In [1]:
# Util Functions
import time
import requests
from urllib.parse import urlparse
import sys
import json
import lxml.html
import csv
from wayback import WaybackClient, memento_url_data
import itertools
from utils import make_request, parse_html, make_link_absolute, page_grab

REQUEST_DELAY = 0.5

In [2]:
def page_grab_old(url):
    urls = []
    response = make_request(url)
    root = parse_html(response.text)
    return (root, url)

In [3]:
# Institute for Fox News


def crawl_fox(base_page, article=set(), video=set()):
    """
    From an initial API query page, run through all possible
    API queries-- putting articles and videos on the pages into
    a set.

    Returns:
    Set of articles and videos
    """
    response = page_grab(base_page)
    json_data = json.loads(response[0].text)
    json_data
    for i in json_data:
        url = make_link_absolute(i["url"], "https://www.foxnews.com/politics")
        if url.startswith("https://www.foxnews.com/politics"):
            article.add(url)
        else:
            video.add(url)
    begin = base_page.find("from") + 5
    end = base_page.find("&media")
    articlenumber = int(base_page[begin:end])
    if articlenumber < 9970:
        articlenumber += 30
        articlenumber = str(articlenumber)
        rev_basepage = (
            base_page[0:begin] + articlenumber + base_page[end : (len(base_page) + 1)]
        )
        crawl_fox(rev_basepage, article, video)
    return article, video


article, video = crawl_fox(
    "https://www.foxnews.com/api/article-search?searchBy=categories&values=fox-news%2Fpolitics&size=30&from=15&mediaTags=primary_politics"
)

article, video

Fetching https://www.foxnews.com/api/article-search?searchBy=categories&values=fox-news%2Fpolitics&size=30&from=15&mediaTags=primary_politics
Fetching https://www.foxnews.com/api/article-search?searchBy=categories&values=fox-news%2Fpolitics&size=30&from=45&mediaTags=primary_politics
Fetching https://www.foxnews.com/api/article-search?searchBy=categories&values=fox-news%2Fpolitics&size=30&from=75&mediaTags=primary_politics
Fetching https://www.foxnews.com/api/article-search?searchBy=categories&values=fox-news%2Fpolitics&size=30&from=105&mediaTags=primary_politics
Fetching https://www.foxnews.com/api/article-search?searchBy=categories&values=fox-news%2Fpolitics&size=30&from=135&mediaTags=primary_politics
Fetching https://www.foxnews.com/api/article-search?searchBy=categories&values=fox-news%2Fpolitics&size=30&from=165&mediaTags=primary_politics
Fetching https://www.foxnews.com/api/article-search?searchBy=categories&values=fox-news%2Fpolitics&size=30&from=195&mediaTags=primary_politics
Fe

KeyboardInterrupt: 

In [24]:
from wayback import WaybackClient, memento_url_data
import itertools

client = WaybackClient()
results = client.search("https://www.foxnews.com/politics", match_type="exact")
first10 = list(itertools.islice(results, 50000))
post2015 = []
for i in range(len(first10)):
    if first10[i].timestamp.year >= 2015:
        post2015.append(first10[i].view_url)
post2015
# memento_url_data(record.raw_url)

['https://web.archive.org/web/20150101055328/http://www.foxnews.com/politics',
 'https://web.archive.org/web/20150101055606/http://www.foxnews.com/politics/',
 'https://web.archive.org/web/20150101190029/http://www.foxnews.com/politics/',
 'https://web.archive.org/web/20150101203356/http://www.foxnews.com/politics/',
 'https://web.archive.org/web/20150101220011/http://www.foxnews.com/politics',
 'https://web.archive.org/web/20150102010527/http://www.foxnews.com/politics',
 'https://web.archive.org/web/20150102010955/http://www.foxnews.com/politics/',
 'https://web.archive.org/web/20150102080640/http://www.foxnews.com/politics',
 'https://web.archive.org/web/20150102080843/http://www.foxnews.com/politics/',
 'https://web.archive.org/web/20150103084335/http://www.foxnews.com/politics/',
 'https://web.archive.org/web/20150103092206/http://www.foxnews.com/politics',
 'https://web.archive.org/web/20150103101110/http://www.foxnews.com/politics',
 'https://web.archive.org/web/20150103101321/h

In [25]:
length = len(post2015)

21306

In [26]:
url = "https://www.foxnews.com/politics"


def get_urls(url, session=None):
    """
    This function takes a URLs and returns lists of URLs
    for containing each article on that page.

    Parameters:
        * url:  a URL to a page of articles

    Returns:
        A list of article URLs on that page.
    """
    response = page_grab(url, session)
    urls = []
    container = response[0].cssselect("article")

    for j in container:
        atr = j.cssselect("a")
        if atr and len(atr) > 0:
            href = atr[0].get("href")
            if len(href) > 0:
                urls.append(
                    make_link_absolute(href, "https://www.foxnews.com/politics")
                )
    return urls


def crawl_wayback_fox(url):
    client = WaybackClient()
    results = client.search(url, match_type="exact")
    politics_sec_copies = list(itertools.islice(results, 30000))
    post2015 = []
    for i in range(len(politics_sec_copies)):
        if politics_sec_copies[i].timestamp.year >= 2015:
            articles = get_urls(politics_sec_copies[i].view_url, client)
            post2015 += articles
    filtered = list(
        filter(lambda x: "http://www.foxnews.com/politics/20" in x, post2015)
    )

    # converts archive links back to current article links
    orig_artlinks = []
    for a in filtered:
        orig_artlinks.append(memento_url_data(a)[0])
    return orig_artlinks


archived = crawl_wayback_fox(url)

Fetching https://web.archive.org/web/20150101055328/http://www.foxnews.com/politics
Fetching https://web.archive.org/web/20150101055606/http://www.foxnews.com/politics/
Fetching https://web.archive.org/web/20150101190029/http://www.foxnews.com/politics/
Fetching https://web.archive.org/web/20150101203356/http://www.foxnews.com/politics/
Fetching https://web.archive.org/web/20150101220011/http://www.foxnews.com/politics
Fetching https://web.archive.org/web/20150102010527/http://www.foxnews.com/politics
Fetching https://web.archive.org/web/20150102010955/http://www.foxnews.com/politics/
Fetching https://web.archive.org/web/20150102080640/http://www.foxnews.com/politics
Fetching https://web.archive.org/web/20150102080843/http://www.foxnews.com/politics/
Fetching https://web.archive.org/web/20150103084335/http://www.foxnews.com/politics/
Fetching https://web.archive.org/web/20150103092206/http://www.foxnews.com/politics
Fetching https://web.archive.org/web/20150103101110/http://www.foxnews

ConnectionError: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20150305220131/http://www.foxnews.com/politics (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x11b6c3ee0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
def crawl_wayback_fox(url, num_pages):
    session = WaybackSession()
    client = WaybackClient(session)
    results = client.search(url, match_type="exact")

    for i in range(num_pages):
        politics_sec_copies = next(results)
        if politics_sec_copies.timestamp.year >= 2015:
            articles = get_urls(politics_sec_copies.view_url, session)
            post2015 += articles
    filtered = list(
        filter(lambda x: "http://www.foxnews.com/politics/20" in x, post2015)
    )

    # converts archive links back to current article links
    orig_artlinks = []
    for a in filtered:
        orig_artlinks.append(memento_url_data(a)[0])
    return orig_artlinks


archived = crawl_wayback_fox(url)

In [ ]:
# save data via json
json_data = json.dumps(list(article))
json_data


# save as csv
def create_csv(set1, set2, filename):
    """
    turns list of articles and videos into a csv with
    these values as respective columns.
    """
    with open(filename, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Articles", "Video"])
        max_length = max(len(set1), len(set2))
        for i in range(max_length):
            row = [
                list(set1)[i] if i < len(set1) else "",
                list(set2)[i] if i < len(set2) else "",
            ]
            writer.writerow(row)


# Calling the function to create the CSV file
create_csv(article, video, "fox_articles.csv")

In [21]:
client = WaybackClient()
results = client.search(url, match_type="exact")
politics_sec_copies = list(itertools.islice(results, 30000))

In [22]:
for i in range(len(politics_sec_copies)):
    if politics_sec_copies[i].timestamp.year >= 2015:
        articles = get_urls(politics_sec_copies[i].view_url)
        post2015 += articles
    filtered = list(
        filter(lambda x: "http://www.foxnews.com/politics/20" in x, post2015)
    )

    # converts archive links back to current article links
    orig_artlinks = []
    for a in filtered:
        orig_artlinks.append(memento_url_data(a)[0])
orig_artlinks

Fetching https://web.archive.org/web/20150101055328/http://www.foxnews.com/politics
Fetching https://web.archive.org/web/20150101055606/http://www.foxnews.com/politics/
Fetching https://web.archive.org/web/20150101190029/http://www.foxnews.com/politics/
Fetching https://web.archive.org/web/20150101203356/http://www.foxnews.com/politics/
Fetching https://web.archive.org/web/20150101220011/http://www.foxnews.com/politics
Fetching https://web.archive.org/web/20150102010527/http://www.foxnews.com/politics
Fetching https://web.archive.org/web/20150102010955/http://www.foxnews.com/politics/
Fetching https://web.archive.org/web/20150102080640/http://www.foxnews.com/politics
Fetching https://web.archive.org/web/20150102080843/http://www.foxnews.com/politics/
Fetching https://web.archive.org/web/20150103084335/http://www.foxnews.com/politics/
Fetching https://web.archive.org/web/20150103092206/http://www.foxnews.com/politics
Fetching https://web.archive.org/web/20150103101110/http://www.foxnews

ConnectionError: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20150124234708/http://www.foxnews.com/politics/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x11cfc9090>: Failed to establish a new connection: [Errno 61] Connection refused'))